In [140]:
import os
os.getcwd()

'D:\\Mallikarjun\\UPXAcademylearning\\Jupyter\\DataScience\\TensorFlow-Deep_Learning'

In [141]:
import tensorflow as tf

In [142]:
tf.reset_default_graph()

### Step 1 Read data 

#### Use MNIST image dataset provided by tensflow library

## Step -1 Load the dataset

In [143]:

from tensorflow.examples.tutorials.mnist import input_data as ipdata
mnist=ipdata.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [144]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [145]:
# in the examples above, the dataset file has 2 datasets, Train and Testt
# in reality this will not be case. We need split the dataset into train and test

In [146]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000438D626EF0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000438D626DD8>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000438D626E48>)

In [147]:
## The Train dataset two arrays
## mnist.train.images is an 2D array of 55000 Images, each of the size 784 pixes (28*28) 
## mnist.train.lables is an column vector of 55000 lables, each one in representing anyone of 0- 9 digits
## In reality we may not have a dataset in this nature. We need to create the vector of labels

In [148]:
trainX=mnist.train.images
trainY=mnist.train.labels

In [149]:
print('Traning input features',trainX.shape)
print('Traning Target features',trainY.shape)

Traning input features (55000, 784)
Traning Target features (55000, 10)


In [150]:
testX=mnist.test.images
testY=mnist.test.labels

In [151]:
print('Traning input features',testX.shape)
print('Traning Target features',testY.shape)

Traning input features (10000, 784)
Traning Target features (10000, 10)


## Step 2 assign the intial values for the weights and bais

## 2a intial the parameter to save the log and model

In [152]:
#directory to save the log files, graph
logs_path='/temp/mnist/lr'

## Learning rate
learning_rate=0.03

## no of features for which the model to be trained which is 784
n_features=trainX.shape[1]


## no of outcomes or results 10
n_classes=trainY.shape[1]

## name of the model for stroage
model_name='mnist.ckpt'

#iteration or number of times the data will be passed to model for traning
training_epochs=60


## Step 3 Build the Graph

### 3a Define the placeholder for input variables

In [153]:
with tf.name_scope("inputs"):
    # input data, we are taking all the rows. And has 784 features
    x=tf.placeholder(tf.float32,shape=[None,n_features],name='x-input')
    # outcome, it will all the rows and , 10 classes
    y_=tf.placeholder(tf.float32,shape=[None,n_classes],name='y-input')

### 3b Define the values for wieghts and bais 

In [154]:
# y1=X.w1 + b1 # w1 here is 784 *1 matrix becos it is the weight required for 1st outcome and applied to all the 784 features
# y2=X.w2 + b2 # w1 here is 784 *1 matrix becos it is the weight required for 2nd outcome and applied to all the 784 features
# y3=X.w3 + b3 # w1 here is 784 *1 matrix becos it is the weight required for 3rd outcome and applied to all the 784 features
# In genral w is a matrx of 784 (features) * 10(outcomes)
# b is a vector of 10 outcomes as there is on bais for each out come
# In general y is matrix of #this is 55k *10 . Matrix mutiplication of [55K by 784] X [784 by 10] results in a matrix of 55K by 10 matrix


In [155]:
with tf.name_scope("wieghts"):
    # There are 784 by 10 coumns as we need wieght for each feature
    W=tf.Variable(tf.zeros([n_features,n_classes])) #this is 784 *10 
    b=tf.Variable(tf.zeros([n_classes])) #this is vector of 10 outcomes
    print ("W shape",W.shape)
    print ("b shape",b.shape)

W shape (784, 10)
b shape (10,)


### 3c Predication interms of Probablity - Call the Softmaxx function to get the probabilistic result

In [156]:
### Softmax function
#Y=softmax(X.w + b)
#X is the input data, a matrix of 10000*784
#w is wieghts, a vector 10000*9 (0 to 9 numberz)
#b is the bias

In [157]:
# Do we need normalization. Nopee as number are between 0-255. So the scale is same for all the ddta points

In [158]:
with tf.name_scope("output"):
    y=tf.nn.softmax(tf.matmul(x,W)+b)

y

<tf.Tensor 'output/Softmax:0' shape=(?, 10) dtype=float32>

### 3d Calculate the loss(error) and reduce it

In [159]:
with tf.name_scope("loss"):
    #cross entropy loss
    cross_entropy=tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

### 3e Gradient optimizer

In [160]:
with tf.name_scope("train"):
    train_op=tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
train_op

<tf.Operation 'train/GradientDescent' type=NoOp>

### Model accuracy

In [161]:
with tf.name_scope("Accuracy"):
    prediction=tf.argmax(y,1,name="Predict")
    print ("prediction",prediction)
    correct_prediction=tf.equal(prediction,tf.argmax(y_,1))
    print ("correct_prediction",correct_prediction)
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuarcy")
    

prediction Tensor("Accuracy/Predict:0", shape=(?,), dtype=int64)
correct_prediction Tensor("Accuracy/Equal:0", shape=(?,), dtype=bool)


### Logging and Accuracy

In [162]:
#Train loss and accuracy
training_loss=tf.summary.scalar("training_loss",cross_entropy)
training_accuracy=tf.summary.scalar("training_accuracy",accuracy)

#Test loss and accurach
test_loss=tf.summary.scalar("training_loss",cross_entropy)
test_accuracy=tf.summary.scalar("training_accuracy",accuracy)

writer=tf.summary.FileWriter(logs_path,graph=tf.get_default_graph())


## ExecuteGraph

In [166]:
#start the session
with tf.Session() as sess:
    #intializing the variables 
    sess.run(tf.global_variables_initializer())
    # perform the training cycles
    for epoch in range(training_epochs):
        _,acc,loss=sess.run([train_op,training_accuracy,training_loss],feed_dict={x:trainX,y_:trainY})

        #log the summary
        writer.add_summary(acc,epoch)
        writer.add_summary(loss,epoch)
        
        acc,loss=sess.run([test_accuracy,test_loss],feed_dict={x:testX,y_:testY})
        
        writer.add_summary(acc,epoch)
        writer.add_summary(loss,epoch)
        #for every 10 rounds of training
        if (epoch%5):
            print("epoch",epoch)
            print("Test Accuracy",accuracy.eval(feed_dict={x:testX,y_:testY}))
            
    #Create a Saver to save the graph
    saver = tf.train.Saver()
    saver.save(sess, logs_path + '/' + model_name)    

epoch 1
Test Accuracy 0.6815
epoch 2
Test Accuracy 0.6908
epoch 3
Test Accuracy 0.698
epoch 4
Test Accuracy 0.7051
epoch 6
Test Accuracy 0.7196
epoch 7
Test Accuracy 0.7253
epoch 8
Test Accuracy 0.7289
epoch 9
Test Accuracy 0.7348
epoch 11
Test Accuracy 0.7424
epoch 12
Test Accuracy 0.7459
epoch 13
Test Accuracy 0.7488
epoch 14
Test Accuracy 0.7516
epoch 16
Test Accuracy 0.7557
epoch 17
Test Accuracy 0.7578
epoch 18
Test Accuracy 0.7597
epoch 19
Test Accuracy 0.7612
epoch 21
Test Accuracy 0.767
epoch 22
Test Accuracy 0.77
epoch 23
Test Accuracy 0.7737
epoch 24
Test Accuracy 0.7752
epoch 26
Test Accuracy 0.7801
epoch 27
Test Accuracy 0.7826
epoch 28
Test Accuracy 0.7838
epoch 29
Test Accuracy 0.7853
epoch 31
Test Accuracy 0.7885
epoch 32
Test Accuracy 0.7903
epoch 33
Test Accuracy 0.7923
epoch 34
Test Accuracy 0.7935
epoch 36
Test Accuracy 0.7953
epoch 37
Test Accuracy 0.7969
epoch 38
Test Accuracy 0.7977
epoch 39
Test Accuracy 0.7985
epoch 41
Test Accuracy 0.8009
epoch 42
Test Accuracy